Load Data
  ↓
Text Preprocessing
  ↓
Job Description Preparation
  ↓
TF-IDF Feature Extraction
  ↓
Cosine Similarity
  ↓
Resume Ranking
  ↓
Explainability (Matched / Missing Skills)
  ↓
ATS-Style Shortlisting
  ↓
Failure Analysis


# STEP 1: LOADING & UNDERSTANDING THE DATASET

In [1]:
import pandas as pd


# Load dataset
file_path = "/content/power_bi_resumes_500 (1).xlsx"
df = pd.read_excel(file_path)

# View first 5 rows
df.head()


,resume_id,resume_text
0,R001,"Skills: Power BI Desktop, DAX Queries, SQL Ser..."
1,R002,"Skills: Power BI Desktop, DAX Queries, SQL Ser..."
2,R003,"Skills: Power BI, DAX, SQL, Power Query, SSRS,..."
3,R004,"Skills: Power BI Desktop, DAX Queries, SQL Ser..."
4,R005,"Skills: Power BI Desktop, DAX Queries, SQL Ser..."


In [2]:
df.shape


(500, 2)

In [3]:
df.columns


Index(['resume_id', 'resume_text'], dtype='object')

In [4]:
df.isnull().sum()


,0
resume_id,0
resume_text,0


In [5]:
df['text_length'] = df['resume_text'].apply(len)
df[['resume_id', 'text_length']].head()


,resume_id,text_length
0,R001,466
1,R002,467
2,R003,461
3,R004,467
4,R005,466


# TEXT PREPROCESSING

In [6]:
import re


In [7]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters & numbers (keep words)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text



In [8]:
df['clean_resume_text'] = df['resume_text'].apply(clean_text)


In [9]:
df[['resume_text', 'clean_resume_text']].head(3)


,resume_text,clean_resume_text
0,"Skills: Power BI Desktop, DAX Queries, SQL Ser...",skills power bi desktop dax queries sql server...
1,"Skills: Power BI Desktop, DAX Queries, SQL Ser...",skills power bi desktop dax queries sql server...
2,"Skills: Power BI, DAX, SQL, Power Query, SSRS,...",skills power bi dax sql power query ssrs data ...


# TF-IDF FEATURE EXTRACTION

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
job_description ="""
We are looking for Power BI developers to join our team! In this position, you will be in charge of developing and updating BI reports as well as communicating actionable insights to improve corporate decision-making. You should be able to manage data, evaluate its results, and display them strategically by using visualization tools, DAX queries, charts, and relationships. This will allow an organization to prosper.

This developer's position necessitates extensive understanding of modeling, databases, data warehousing, data integration, and technical elements of business intelligence technologies. You should also be able to read and evaluate product, market, and share trends. You should also have a strong technical understanding of business trends. Strong communication, organizational, and analytical abilities are required.


Roles and Responsibilities

* Formulate automated reports and dashboards using Power BI and other reporting tools.
* Understand business requirements to set functional specifications for reporting applications.
* You should be familiar with SSRS and TSQL, Power Query, MDX, PowerBI, and DAX are just a few of the tools and systems on the MS SQL Server BI Stack.
* Exhibit a foundational understanding of database concepts such relational database architecture, multidimensional database design, and more
* Design data models that transform raw data into insightful knowledge by understanding business requirements in the context of BI.
* Develop technical specifications from business needs, and choose a deadline for work completion.
* Make charts and data documentation that includes descriptions of the techniques, parameters, models, and relationships.
* Developing Power BI desktop to create dashboards, KPI scorecards, and visual reports.
* Establish row-level security on data and comprehend Power BI's application security layer models.
* Examine, comprehend, and study business needs as they relate to business intelligence.
* Design and map data models to transform raw data into insightful information.
* Create dynamic and eye-catching dashboards and reports using Power BI.
* Make necessary tactical and technological adjustments to enhance current business intelligence systems
* Integrate data, alter data, and connect to data sources for business intelligence.


Requirements and Skills

* Extremely good communication skills are necessary to effectively explain the requirements between both internal teams and client teams.
* Exceptional analytical thinking skills for converting data into illuminating reports and reports.
* BS in computer science or information system along with work experience in a related field
* knowledge of data warehousing, data gateway, and data preparation projects
* Working knowledge of Power BI, SSAS, SSRS, and SSIS components of the Microsoft Business Intelligence Stack
* Articulating, representing, and analyzing solutions with the team while documenting, creating, and modeling them.
* Familiarity with the tools and technologies used by the Microsoft SQL Server BI Stack, including SSRS and TSQL, Power Query, MDX, PowerBI, and DAX.
* Knowledge of executing DAX queries on the Power BI desktop.
* Comprehensive understanding of data modeling, administration, and visualization.
* Capacity to perform in an atmosphere where agility and continual development are prioritized.
* Detailed knowledge and understanding of database management systems, OLAP, and the ETL (Extract, Transform, Load) framework.
* Awareness of BI technologies (e.g., Microsoft Power BI, Oracle BI).
* Expertise of SQL queries, SSRS, and SQL Server Integration Services (SSIS)."""

In [12]:
clean_job_description = clean_text(job_description)


In [13]:
documents = [clean_job_description] + df['clean_resume_text'].tolist()


In [14]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000
)

tfidf_matrix = tfidf.fit_transform(documents)


# COSINE SIMILARITY

In [15]:
job_vector = tfidf_matrix[0]
resume_vectors = tfidf_matrix[1:]





In [16]:
similarity_scores = cosine_similarity(job_vector, resume_vectors)[0]


# RESUME RANKING

In [17]:
df['similarity_score'] = similarity_scores


In [18]:
ranked_df = df.sort_values(by='similarity_score', ascending=False)



In [19]:
ranked_df[['resume_id', 'similarity_score']].head(10)


,resume_id,similarity_score
196,R197,0.201919
193,R194,0.201919
194,R195,0.201919
195,R196,0.201919
192,R193,0.201919
26,R027,0.201919
29,R030,0.201919
28,R029,0.201919
76,R077,0.201919
77,R078,0.201919


In [20]:
ranked_df['rank'] = range(1, len(ranked_df) + 1)


In [21]:
final_output = ranked_df[['rank', 'resume_id', 'similarity_score']].head(20)
final_output


,rank,resume_id,similarity_score
196,1,R197,0.201919
193,2,R194,0.201919
194,3,R195,0.201919
195,4,R196,0.201919
192,5,R193,0.201919
26,6,R027,0.201919
29,7,R030,0.201919
28,8,R029,0.201919
76,9,R077,0.201919
77,10,R078,0.201919


# SKILL MATCHING


In [22]:
important_skills = [
    "power bi",
    "dax",
    "sql",
    "sql server",
    "power query",
    "ssrs",
    "ssas",
    "ssis",
    "data modeling",
    "data warehousing",
    "etl",
    "business intelligence"
]


In [23]:
def skill_match_analysis(text, skills):
    matched = []
    missing = []
    for skill in skills:
        if skill in text:
            matched.append(skill)
        else:
            missing.append(skill)
    return matched, missing




In [24]:
top_resumes = ranked_df.head(10).copy()

top_resumes['matched_skills'] = top_resumes['clean_resume_text'].apply(
    lambda x: skill_match_analysis(x, important_skills)[0]
)

top_resumes['missing_skills'] = top_resumes['clean_resume_text'].apply(
    lambda x: skill_match_analysis(x, important_skills)[1]
)

top_resumes[['rank', 'resume_id', 'similarity_score',
             'matched_skills', 'missing_skills']]



,rank,resume_id,similarity_score,matched_skills,missing_skills
196,1,R197,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
193,2,R194,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
194,3,R195,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
195,4,R196,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
192,5,R193,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
26,6,R027,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
29,7,R030,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
28,8,R029,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
76,9,R077,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."
77,10,R078,0.201919,"[power bi, dax, sql, power query, ssrs, data m...","[sql server, ssas, ssis, etl, business intelli..."


In [25]:
high_threshold = ranked_df['similarity_score'].quantile(0.90)
mid_threshold  = ranked_df['similarity_score'].quantile(0.70)


In [26]:
def shortlist_category(score):
    if score >= high_threshold:
        return "Strong Match"
    elif score >= mid_threshold:
        return "Moderate Match"
    else:
        return "Weak Match"




In [27]:
ranked_df['shortlist_category'] = ranked_df['similarity_score'].apply(shortlist_category)

ranked_df['shortlist_category'].value_counts()


,count
shortlist_category,
Weak Match,300
Moderate Match,114
Strong Match,86


In [28]:
final_output = ranked_df[
    ['rank', 'resume_id', 'similarity_score', 'shortlist_category']
]

final_output.head(20)


,rank,resume_id,similarity_score,shortlist_category
196,1,R197,0.201919,Strong Match
193,2,R194,0.201919,Strong Match
194,3,R195,0.201919,Strong Match
195,4,R196,0.201919,Strong Match
192,5,R193,0.201919,Strong Match
26,6,R027,0.201919,Strong Match
29,7,R030,0.201919,Strong Match
28,8,R029,0.201919,Strong Match
76,9,R077,0.201919,Strong Match
77,10,R078,0.201919,Strong Match


In [29]:
final_output.to_csv("final_ranked_resumes.csv", index=False)
